In [1]:
import pandas as pd

In [83]:
df_in = pd.read_csv('../../data_v3/hr_il_tagged.csv')
df_gt = pd.read_csv('../../data_v3/hr_ground_truth.csv')

In [99]:
df = df_in[["event_with_roles","request_method_call","starting_frame_number","selective_filter_data","origin_selective_filter_data", "stream_index", "real_activity_action",]]


In [101]:
df[df["real_activity_action"]=="Activity End"]

,event_with_roles,request_method_call,starting_frame_number,selective_filter_data,origin_selective_filter_data,stream_index,real_activity_action
14,Odoo Application->End Point (HR Manager): [Htt...,1224,183.0,IsNumber,hr.applicant_create,13,Activity End
64,Odoo Application->End Point (HR Manager): [Htt...,faultCode,1121.0,faultCode,hr.applicant_archive_applicant,25,Activity End
79,Odoo Application->End Point (HR Manager): [Htt...,1225,1378.0,IsNumber,hr.applicant_create,34,Activity End
126,Odoo Application->End Point (HR Manager): [Htt...,1,1747.0,IsNumber,hr.applicant_write_stage_id_2,44,Activity End
141,Odoo Application->End Point (HR Manager): [Htt...,1226,2522.0,IsNumber,hr.applicant_create,54,Activity End
226,Odoo Application->End Point (HR Manager): [Htt...,faultCode,4379.0,faultCode,hr.applicant_archive_applicant,89,Activity End
247,Odoo Application->End Point (HR Manager): [Htt...,1,3921.0,IsNumber,hr.applicant_write_stage_id_3,86,Activity End
248,Odoo Application->End Point (HR Manager): [Htt...,1227,4636.0,IsNumber,hr.applicant_create,98,Activity End
311,Odoo Application->End Point (HR Manager): [Htt...,faultCode,6059.0,faultCode,hr.applicant_archive_applicant,123,Activity End
338,Odoo Application->End Point (HR Manager): [Htt...,faultCode,6786.0,faultCode,hr.applicant_archive_applicant,132,Activity End


In [103]:
df_in[df_in["real_activity_action"] == "NoAction"][["event_with_roles","resolved_activities"]].value_counts()

event_with_roles                                                                                                         resolved_activities                 
Odoo Application->db Server/Mail Server: [PgsqlRequest:Simple query:UPDATE:['mail_message_res_partner_needaction_rel']]  ('ResumeReviewActivity',)               64
Odoo Application->db Server/Mail Server: [PgsqlRequest:Simple query:UPDATE:['mail_message']]                             ('ResumeReviewActivity',)               62
Odoo Application->db Server/Mail Server: [PgsqlRequest:Simple query:UPDATE:['hr_applicant']]                             ('ResumeReviewActivity',)               56
Odoo Application->End Point (HR Manager): [HttpResponse:HTTP/1.0 200 OK\r\n]                                             ('GenerateJobApplicationActivity',)     51
Odoo Application->db Server/Mail Server: [PgsqlRequest:Simple query:UPDATE:['ir_attachment']]                            ('ResumeReviewActivity',)               50
                      

The end events do not differ between different activities. -> Different resolving needed 

In [113]:
df["ele_counts"] = df.groupby("real_activity_action").cumcount()+1
df["start_counts"] = df.apply(lambda row: row["ele_counts"] if row["real_activity_action"]== "Activity Start" else 0 , axis = 1).replace(0,None).ffill()
df["end_counts"] = df.apply(lambda row: row["ele_counts"] if row["real_activity_action"] == "Activity End" else 0 , axis = 1).replace(0,None).ffill().fillna(0).astype(int)
df["open_activities"] = df["start_counts"] - df["end_counts"]

/var/folders/qt/gt7q1tn978x_02pv47b2lrvm0000gn/T/ipykernel_66560/2140098044.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["ele_counts"] = df.groupby("real_activity_action").cumcount()+1
/var/folders/qt/gt7q1tn978x_02pv47b2lrvm0000gn/T/ipykernel_66560/2140098044.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["start_counts"] = df.apply(lambda row: row["ele_counts"] if row["real_activity_action"]== "Activity Start" else 0 , axis = 1).replace(0,None).ffill()
/var/folders/qt/gt7q1tn978x_02pv47b2

 Based on the amount of start / end events we can assign sequences directly.
 When a new event is registered we have the following options
- Response package: Map it by packet it is referring to (via starting frame index) 
- Other packets: Check likelihood that is belongs to each currently open stacks.  (Based on clustering? ) 

For each arriving packet do: 

1. Classify Activity Action
2. Based on activity action do the following
    - Activity Start
    Open new Stack
    - Activity End
    Assign to a stack, *Close and emit the stack*
    - No Action
    Assign to a stack
3. Assigning to stacks
- Response packets have a starting frame number, where the request originated. We can assign to the same stack as the origin event
- Each end event is a response. -> No need to assign end events 

**We are only left with assigning No Action events**
- Most of the time we will have one sequence running for a bit. The other one fresh
- Ideas: Check positioning of events in sequences
- Classifier whether one event continues a sequence
- Stream index? 


In [114]:
df

,event_with_roles,request_method_call,starting_frame_number,selective_filter_data,origin_selective_filter_data,stream_index,real_activity_action,ele_counts,start_counts,end_counts,open_activities
0,End Point (HR Manager)->Odoo Application: [Htt...,version,NaN,version,NaN,1,Activity Start,1,1,0,1
1,Odoo Application->End Point (HR Manager): [Htt...,server_version,17.0,NaN,version,1,NoAction,1,1,0,1
2,End Point (HR Manager)->Odoo Application: [Htt...,authenticate,NaN,NaN,NaN,5,NoAction,2,1,0,1
3,Odoo Application->db Server/Mail Server: [Pgsq...,NaN,NaN,NaN,NaN,8,NoAction,3,1,0,1
4,Odoo Application->End Point (HR Manager): [Htt...,6,57.0,IsNumber,NaN,5,NoAction,4,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...
1293,Odoo Application->db Server/Mail Server: [Pgsq...,NaN,NaN,NaN,NaN,12,NoAction,1221,37,36,1
1294,Odoo Application->End Point (Recruiting manage...,1,27852.0,IsNumber,hr.applicant_write_stage_id_8,535,NoAction,1222,37,36,1
1295,End Point (Recruiting manager)->Odoo Applicati...,execute_kw,NaN,hr.applicant_archive_applicant,NaN,538,NoAction,1223,37,36,1
1296,Odoo Application->db Server/Mail Server: [Pgsq...,NaN,NaN,NaN,NaN,8,NoAction,1224,37,36,1


In [115]:
# Stream Index Approach

In [116]:
class_test = df.iloc[142:248]

In [110]:
class_test.to_csv("../../data_v3/classtest.csv")